In [ ]:
pip cache purge

Files removed: 2917


In [ ]:
!wget https://object.pouta.csc.fi/OPUS-NLLB/v1/moses/en-sa.txt.zip -O en-sa.txt.zip

--2024-09-09 14:31:30--  https://object.pouta.csc.fi/OPUS-NLLB/v1/moses/en-sa.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146718593 (140M) [application/zip]
Saving to: ‘en-sa.txt.zip’

en-sa.txt.zip       100%[===================>] 139.92M  30.6MB/s    in 5.7s    

2024-09-09 14:31:36 (24.6 MB/s) - ‘en-sa.txt.zip’ saved [146718593/146718593]



In [ ]:
!unzip en-sa.txt.zip -d en-sa

Archive:  en-sa.txt.zip
  inflating: en-sa/README            
  inflating: en-sa/LICENSE           
  inflating: en-sa/NLLB.en-sa.en     
  inflating: en-sa/NLLB.en-sa.sa     
  inflating: en-sa/NLLB.en-sa.scores  


In [ ]:
# Load the English sentences
with open('en-sa/NLLB.en-sa.en', 'r', encoding='utf-8') as f:
    en_lines = f.readlines()

# Load the Sanskrit sentences
with open('en-sa/NLLB.en-sa.sa', 'r', encoding='utf-8') as f:
    sa_lines = f.readlines()

# Verify that the lines match
assert len(en_lines) == len(sa_lines), "The number of lines in the English and Sanskrit files do not match!"

# Print a sample pair to check
print(f"English: {en_lines[10]}")
print(f"Sanskrit: {sa_lines[10]}")

English: To whom He pleases,

Sanskrit: इच्छां गृह्णाति यो नित्यं करुणारसनिर्भरः



In [ ]:
!pip uninstall pyarrow

Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pyarrow-14.0.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pyarrow/*
Proceed (Y/n)? n


In [ ]:
!pip install pyarrow

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install transformers sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
# Create a DataFrame with the English and Sanskrit pairs
from datasets import Dataset
import pandas as pd
df = pd.DataFrame({
    'translation': [{"en": en.strip(), "sa": sa.strip()} for en, sa in zip(en_lines, sa_lines)]
})

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Check the dataset
print(dataset)
print(dataset[0])

# Check the type and keys of the dataset
print(type(dataset))
print(dataset.column_names)  # If using Hugging Face datasets

In [ ]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import random
import re
from tqdm import tqdm
import gc
from torch.utils.data import Dataset as TorchDataset, DataLoader, TensorDataset
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import torch
import numpy as np

In [ ]:
# 1. Create a smaller subset of the data
def create_subset(dataset, size=40000, random_state=42):
    random.seed(random_state)
    subset_indices = random.sample(range(len(dataset)), size)
    return dataset.select(subset_indices)

# Create a subset of 10,000 samples
subset_size = 40000
subset = dataset.shuffle(seed=42).select(range(subset_size))
print(f"Subset size: {len(subset)}")
print(subset[0])

Subset size: 40000
{'translation': {'en': 'The track on the map', 'sa': 'नक़्शे पर Vejle'}}


In [ ]:
!pip install dask distributed

In [ ]:
# 2. Preprocess the data
import unicodedata
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import dask.dataframe as dd

def normalize_text(text):
    # Normalize unicode characters
    text = unicodedata.normalize('NFKC', text)
    return text

def remove_special_characters(text):
    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)
    return text

def tokenize_and_stem(text):
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

def preprocess_sanskrit(text):
    # Apply Sanskrit-specific preprocessing
    text = remove_special_characters(text)
    # Additional Sanskrit-specific rules can be added here
    return text

def preprocess_text(text, language='en'):
    if isinstance(text, str):
        text = text.strip().lower()
        if language == 'sa':  # Sanskrit specific preprocessing
            text = normalize_text(text)
            text = remove_special_characters(text)
            # Additional Sanskrit-specific preprocessing can be added here
    else:
        # Handle unexpected input (e.g., None, empty string)
        return ''
    return text

In [ ]:
# en_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
# sa_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MBartForConditionalGeneration, MBart50TokenizerFast

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-sa-en")
# tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# # Tokenization function
# # def tokenize_pair(example, max_length=128):
# #     en_text = example['translation']['en']
# #     sa_text = example['translation']['sa']

# #     en_tokens = en_tokenizer.encode(preprocess_text(en_text),
# #                                     max_length=max_length,
# #                                     padding='max_length',
# #                                     truncation=True)
# #     sa_tokens = sa_tokenizer.encode(preprocess_text(sa_text),
# #                                     max_length=max_length,
# #                                     padding='max_length',
# #                                     truncation=True)
# #     return {
# #         'input_ids': sa_tokens,
# #         'attention_mask': [1 if token != sa_tokenizer.pad_token_id else 0 for token in sa_tokens],
# #         'labels': en_tokens
# #     }

# # def tokenize_pair(example, max_length=128):
# #     en_text = example['translation']['en']
# #     sa_text = example['translation']['sa']

# #     inputs = tokenizer(preprocess_text(sa_text),
# #                        max_length=max_length,
# #                        padding='max_length',
# #                        truncation=True,
# #                        return_tensors="pt")

# #     with tokenizer.as_target_tokenizer(): # Remove lang parameter
# #         labels = tokenizer(preprocess_text(en_text),
# #                            max_length=max_length,
# #                            padding='max_length',
# #                            truncation=True,
# #                            return_tensors="pt")

# #     return {
# #         'input_ids': inputs['input_ids'].squeeze(),
# #         'attention_mask': inputs['attention_mask'].squeeze(),
# #         'labels': labels['input_ids'].squeeze()
# #     }
# def tokenize_pair(example, max_length=128):
#     en_text = example['translation']['en']
#     sa_text = example['translation']['sa']

#     if isinstance(en_text, str) and isinstance(sa_text, str):
#         inputs = tokenizer(preprocess_text(sa_text, language='sa'),
#                            max_length=max_length,
#                            padding='max_length',
#                            truncation=True,
#                            return_tensors="pt")

#         with tokenizer.as_target_tokenizer():
#             labels = tokenizer(preprocess_text(en_text, language='en'),
#                                max_length=max_length,
#                                padding='max_length',
#                                truncation=True,
#                                return_tensors="pt")

#         return {
#             'input_ids': inputs['input_ids'].squeeze(),
#             'attention_mask': inputs['attention_mask'].squeeze(),
#             'labels': labels['input_ids'].squeeze()
#         }
#     else:
#         return None

# chunk_size = 4000
# processed_chunks = []

# # for i in tqdm(range(0, len(subset), chunk_size)):
# #     chunk = subset.select(range(i, min(i + chunk_size, len(subset))))
# #     processed_chunk = chunk.map(tokenize_pair, remove_columns=['translation'])
# #     processed_data.extend(processed_chunk)
# #     gc.collect()  # Force garbage collection
# for i in range(0, len(subset), chunk_size):
#     chunk = subset.select(range(i, min(i + chunk_size, len(subset))))
#     processed_chunk = chunk.map(tokenize_pair)
#     processed_chunks.append(processed_chunk)
#     print(f"Processed chunk {i//chunk_size + 1}/{len(subset)//chunk_size + 1}")


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
# Combine processed chunks
from datasets import concatenate_datasets
full_dataset = concatenate_datasets(processed_chunks)

# Verify the combined dataset
print("Number of rows:", full_dataset.num_rows)
print("Features:", full_dataset.features)

Number of rows: 40000
Features: {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [ ]:
# # Convert to PyTorch dataset
# input_ids = torch.tensor([item['input_ids'] for item in processed_data])
# attention_mask = torch.tensor([item['attention_mask'] for item in processed_data])
# labels = torch.tensor([item['labels'] for item in processed_data])

# dataset = TensorDataset(input_ids, attention_mask, labels)

In [ ]:
full_dataset.set_format('torch')
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 36000
Validation dataset size: 4000


In [ ]:
# # Custom dataset class
# class TranslationDataset(TorchDataset):
#     def __init__(self, hf_dataset):
#         self.data = hf_dataset

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]
#         return {
#             'input_ids': torch.tensor(item['input_ids']),
#             'attention_mask': torch.tensor(item['attention_mask']),
#             'labels': torch.tensor(item['labels'])
#         }

# # Create PyTorch datasets
# train_data = TranslationDataset(train_dataset)
# val_data = TranslationDataset(val_dataset)


In [ ]:
# # Model initialization
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
# model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-sa-en")
# model = MarianMTModel.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/295M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# !pip install rouge_score # install the missing rouge_score library
# !pip install nltk
# from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
# from rouge_score import rouge_scorer
# import nltk

In [ ]:
from datasets import load_metric

# Load the BLEU metric
metric = load_metric("sacrebleu")

<ipython-input-25-b66ebf5e7018>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
# Evaluation function
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]

#     decoded_preds = en_tokenizer.batch_decode(preds, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, en_tokenizer.pad_token_id)
#     decoded_labels = en_tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # BLEU score
#     smoothie = SmoothingFunction().method4
#     bleu_score = corpus_bleu([[l.split()] for l in decoded_labels],
#                              [p.split() for p in decoded_preds],
#                              smoothing_function=smoothie)

#     # ROUGE score
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
#     rouge_scores = scorer.score(decoded_labels[0], decoded_preds[0])

#     return {
#         'bleu': bleu_score,
#         'rouge1': rouge_scores['rouge1'].fmeasure,
#         'rouge2': rouge_scores['rouge2'].fmeasure,
#         'rougeL': rouge_scores['rougeL'].fmeasure,
#     }

def compute_metrics(eval_preds):
  preds, labels = eval_preds
  # Decode generated translations
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # SacreBLEU expects a list of references for each translation
  decoded_labels = [[label] for label in decoded_labels]
  # Compute BLEU score
  result = metric.compute(predictions=decoded_preds, references=decoded_labels)
  return {"bleu": result["score"]}

In [ ]:
# Training arguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate after each epoch
    learning_rate=5e-5,           # Learning rate; you can experiment with this
    per_device_train_batch_size=4,  # Low batch size to avoid memory issues
    per_device_eval_batch_size=4,   # Same as training batch size
    weight_decay=0.01,              # Standard weight decay
    save_total_limit=3,             # Limit the number of saved checkpoints
    num_train_epochs=3,             # 3 epochs for initial training
    predict_with_generate=True,     # Important for Seq2Seq models
    fp16=True,                      # Mixed precision training
    push_to_hub=False,              # Avoid pushing to the hub
    gradient_accumulation_steps=8,  # Accumulate gradients to simulate larger batch size
    # eval_steps=1000,
    # logging_steps=200,
    # save_steps=2000,
    # max_grad_norm=0.5,
    optim="adamw_torch",            # Standard optimizer
)


# Trainer initialization
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Start training
trainer.train()

trainer.save_model("./StoE")

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluation
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
# Inference function
def translate(sanskrit_text):
    inputs = sa_tokenizer(preprocess_text(sanskrit_text), return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model.generate(**inputs)
    return en_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
sanskrit_text = "नमस्ते जगत्"
translated_text = translate(sanskrit_text)
print(f"Sanskrit: {sanskrit_text}")
print(f"English translation: {translated_text}")